In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from dhdrnet.util import DATA_DIR, ROOT_DIR

plt.rcParams.update({"font.size": 14})
plt.rcParams.update({"axes.labelsize": 18})


# "test"
figdir = ROOT_DIR / "precomputed_data" / "figures"

In [ ]:
df = (
    pd.read_csv(ROOT_DIR / "precomputed_data" / "store_2020-10-27.csv", index_col=0)
    .drop(columns="ev1")
    .rename(columns={"ev2": "ev"})
)
names = pd.read_csv(ROOT_DIR / "precomputed_data" / "test_current.csv")
df = df[df["name"].isin(names["names"])]
df = df.drop_duplicates()
df

In [ ]:
grouped = df.set_index("name").groupby(["metric"])
grouped.describe()["score"][["mean", "std"]]

In [ ]:
idxmins = (
    df[(df["metric"] == "perceptual") | (df["metric"] == "rmse")]
    .groupby(["name", "metric"])["score"]
    .idxmin()
)
best_scores_min = df.loc[idxmins]
best_scores_min
# this is the minimum score for each metric for each image
# unfortunately we want min for mse and perceptual
# and maxes for psnr, ssim

In [ ]:
idxmaxs = (
    df[(df["metric"] == "psnr") | (df["metric"] == "ssim")]
    .groupby(["name", "metric"])["score"]
    .idxmax()
)
best_scores_max = df.loc[idxmaxs]
best_scores_max

In [ ]:
best_scores_df = pd.concat([best_scores_min, best_scores_max])
oracle = best_scores_df.groupby(["metric"]).describe()["score"][["mean", "std"]]
oracle

In [ ]:
oracle_by_ev = best_scores_df.groupby(["metric", "ev"]).describe()["score"]

for metric in ["rmse", "psnr", "ssim", "perceptual"]:
    plt.plot(
        oracle_by_ev["mean"][metric].index,
        oracle_by_ev["mean"][metric],
    )
    plt.xlabel("EV")
    plt.ylabel(metric.upper())
    plt.savefig(figdir / f"{metric.upper()}_best_scores.pdf", pad_inches=0)
    plt.show()

In [ ]:
df.loc[df["ev"] == 3.5].groupby("metric").describe()["score"][["mean", "std"]]

In [ ]:
df_by_metric = df.groupby(["metric", "ev"]).describe()["score"]
plt.plot(
    df_by_metric["mean"]["rmse"].index,
    df_by_metric["mean"]["rmse"],
    #     s=0.2 * df_by_metric["std"]["rmse"] ** 2,
)
plt.xlabel("EV")
plt.ylabel("RMSE")
plt.show()
plt.savefig(figdir / "RMSE_by_ev.pdf")

In [ ]:
df.loc[df["ev"] == 3.5].groupby("metric").describe()["score"][["mean", "std"]].to_latex(
    figdir / "fixed_choice.tex"
)

In [ ]:
cdf = pd.read_csv(
    ROOT_DIR / "precomputed_data" / "store_updown_2020-10-26.csv"
).drop_duplicates()
cdf.loc[cdf["metric"] == "rmse", "score"] = cdf.loc[
    cdf["metric"] == "rmse", "score"
].apply(np.sqrt)

cdf

In [ ]:
cdf.groupby(["ev", "metric"]).describe()["score"][["mean", "std"]].to_latex(
    figdir / "conv_choice.tex"
)
cdf.groupby(["ev", "metric"]).describe()["score"][["mean", "std"]]

In [ ]:
def parse_preds(pred_str):
    splits = pred_str.split(" ")
    return [float(s) for s in splits[1:-1] if len(s) > 0]


model_name = "mobile"
model_df = pd.read_csv(
    ROOT_DIR / "precomputed_data" / f"{model_name}_preds.csv", index_col=0
)
model_df["pred"] = model_df["pred"].apply(parse_preds)
model_df

In [ ]:
from dhdrnet.util import (
    get_best_preds,
    get_pred,
    get_scores_for_preds,
    get_topk_score_df,
    get_worst_preds,
)

In [ ]:
model_df

In [ ]:
t = get_scores_for_preds(model_df, df)
t.groupby("metric").describe()["score"]  # These are the top-10 accuracies

In [ ]:
t.describe()

In [ ]:
worst_ind = get_worst_preds(model_df, df, metric="psnr", dir="up", n=25)
worst_ind

In [ ]:
best_ind = get_best_preds(model_df, df, metric="rmse", dir="down", n=60)
best_ind

In [ ]:
best_ind.nunique()

In [ ]:
from dhdrnet.util import get_furthest_pred

worst_by_pred = get_furthest_pred(model_df, df, n=25)

In [ ]:
r = get_topk_score_df(t, k=1)
r

In [ ]:
model_by_ev = r.groupby(["metric", "ev"]).describe()["score"]
for metric in ["rmse", "psnr", "ssim", "perceptual"]:
    plt.plot(
        model_by_ev["mean"][metric].index,
        model_by_ev["mean"][metric],
        #         model_by_ev["std"][metric]**2,
    )
    plt.xlabel("EV")
    plt.ylabel(metric.upper())
    # plt.savefig(figdir / f"{model_name}_{metric}_by_ev.pdf")
    plt.show()

In [ ]:
t2 = get_pred(model_df, df)
t2.nunique()

In [ ]:
t2

In [ ]:
t2.groupby("metric").describe()["score"][["mean", "std"]]

In [ ]:
t2.groupby("metric").describe()["score"][["mean", "std"]]

In [ ]:
r.groupby("metric").describe()["score"][["mean", "std"]]

In [ ]:
from dhdrnet.gen_pairs import DataGenerator

gen = DataGenerator(raw_path=DATA_DIR / "dngs", out_path=DATA_DIR)

In [ ]:
sample = r[r["metric"] == "rmse"].sample(n=100)
sample

In [ ]:
for _i, (ev, name, _, score) in best_ind.iterrows():  # .sample(n=4).iterrows():
    baseline_img, pred_img = gen.get_exposures(name, [0.0, ev])
    gt_img = gen.get_ground_truth(name)
    fused_img = gen.get_fused(name, [0.0, ev])
    plt.tight_layout()
    fig, axs = plt.subplots(1, 4, figsize=(18, 6))
    for ax in axs:
        ax.set_xticks([])
        ax.set_yticks([])
    axs[0].imshow(baseline_img[..., [2, 1, 0]])
    axs[0].set_xlabel("EV 0 Image")

    axs[1].imshow(pred_img[..., [2, 1, 0]])
    axs[1].set_xlabel(f"Predicted EV: {ev}")

    axs[2].imshow(fused_img[..., [2, 1, 0]])
    axs[2].set_xlabel("Fused Image")

    axs[3].imshow(gt_img[..., [2, 1, 0]])
    axs[3].set_xlabel("Ground Truth Image")

    plt.savefig(
        figdir / "best_rmse_predictions" / f"{name}_best_rmse_predictions.svg",
        bbox_inches="tight",
        transparent="True",
        pad_inches=0,
    )
    plt.show()

    plt.close("all")